In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy.random as npr
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from keras.optimizers import Adam
from keras_nlp.layers import PositionEmbedding

In [3]:
seed = 428

np.random.seed(seed)
tf.random.set_seed(seed)
random.seed(seed)

In [4]:
def get_masked_input_and_labels(encoded_texts, n_cat):
    # For each sentence, mask each word one-by-one

    encoded_texts_masked = []
    y_labels = []

    for encoded_text in encoded_texts:
        for i in range(len(encoded_text)):
            encoded_text_masked = np.copy(encoded_text)
            y_label = encoded_text_masked[i]
            encoded_texts_masked.append(np.delete(encoded_text_masked, i))
            y_labels.append(np.array([y_label]))

    return np.array(encoded_texts_masked), np.array(y_labels)

In [5]:
#### K = number of countries = number of capitals = number of universities = number of mascots
#### M = number of words only used by each topic
#### S = number of words used by both topics
#### L = sentence length
#### q1, q2 = probability of having 1 or 2 pairs
#### embed_dim = dimension of embeddings
#### n_sentences = number of training sentences

def train_model(K, M, S, L, q1, q2, embed_dim, n_sentences):
    
    countries = ['country_' + str(i) for i in range(K)]
    capitals = ['capital_' + str(i) for i in range(K)]
    universities = ['university_' + str(i) for i in range(K)]
    mascots = ['mascot_' + str(i) for i in range(K)]
    random_capitals = ['random_capital_' + str(i) for i in range(M)]
    random_mascots = ['random_mascot_' + str(i) for i in range(M)]
    randoms = ['random_' + str(i) for i in range(S)]

    vocabs = countries + capitals + universities + mascots + random_capitals + random_mascots + randoms
    vocab_map = {}

    for i in range(len(vocabs)):
        vocab_map[vocabs[i]] = i
        
    sentences = []
    sentences_number = []
    
    q0 = 1 - q1 - q2

    for i in range(n_sentences):

        sentence = []
        
        temp = npr.uniform()
        temp2 = npr.uniform()
        
        if temp2 <= q0:
            n_pairs = 0
        elif temp2 <= q0 + q1:
            n_pairs = 1
        else:
            n_pairs = 2
        
        if temp <= 0.5: ### country - capital
        
            pairs = np.random.choice(np.arange(K), n_pairs, replace = False)
#             pairs2 = np.random.choice(np.arange(M), n_pairs, replace = False)
            for pair in pairs:
                sentence.append(countries[pair])
                sentence.append(capitals[pair])        
#             for pair in pairs2:
#                 temp3 = npr.uniform()
#                 if temp3 <= 0.5:
#                     sentence.append(random_capitals[pair])
#                 else:
#                     sentence.append(random_mascots[pair])
            
            randoms_dup = 2 * random_capitals + 1 * randoms 
            sentence += list(np.random.choice(randoms_dup, L - 2 * n_pairs, replace = False))  
                 
        else: ### university - mascot
            
            pairs = np.random.choice(np.arange(K), n_pairs, replace = False)
#             pairs2 = np.random.choice(np.arange(M), n_pairs, replace = False)
            for pair in pairs:
                sentence.append(universities[pair])
                sentence.append(mascots[pair])        
#             for pair in pairs2:
#                 temp3 = npr.uniform()
#                 if temp3 <= 0.5:
#                     sentence.append(random_mascots[pair])
#                 else:
#                     sentence.append(random_capitals[pair])

            
            randoms_dup = 1 * randoms + 2 * random_mascots
            sentence += list(np.random.choice(randoms_dup, L - 2 * n_pairs, replace = False))  
            
        
        

        #sentence += list(np.random.choice(randoms + random_capitals + random_mascots, L - 2 * n_pairs, replace = False))  

        sentence_number = [vocab_map[i] for i in sentence]
        sentences.append(sentence)
        sentences_number.append(sentence_number)
        
    x_train = np.array(sentences_number)
    n_cat = len(vocab_map)
    x_masked_train, y_masked_labels_train = get_masked_input_and_labels(x_train, n_cat)
    
    callback = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5, restore_best_weights = True)
    inputs = layers.Input((x_masked_train.shape[1],), dtype=tf.int64)
    word_embeddings = layers.Embedding(n_cat, embed_dim, name="word_embedding")(inputs)
    encoder_output = layers.GlobalAveragePooling1D()(word_embeddings)
    mlm_output = layers.Dense(n_cat, name="mlm_cls", activation="softmax", use_bias=False)(encoder_output)
    mlm_model = keras.Model(inputs = inputs, outputs = mlm_output)
    adam = Adam()
    mlm_model.compile(loss='sparse_categorical_crossentropy', optimizer=adam)

    history = mlm_model.fit(x_masked_train, y_masked_labels_train,
                        validation_split = 0.5, callbacks = [callback], 
                        epochs=500, batch_size=128, verbose=0)
    
    return sentences, vocab_map, mlm_model

In [6]:
def get_acc_prob(K, M, S, L, q1, q2, embed_dim, n_sentences, n_samples):
    
    sentences, vocab_map, current_model = train_model(K, M, S, L, q1, q2, embed_dim, n_sentences)

    acc_capitals = []
    prob_capitals = []

    for _ in range(n_samples):
        sentence = []
        random_capitals = np.random.choice(np.arange(K), int(L/2), replace = False)
        for random_capital in random_capitals:
            sentence.append('country_' + str(random_capital))
            sentence.append('capital_' + str(random_capital))
        sentence = sentence[:-1]
        sentence_number = [vocab_map[i] for i in sentence]
        temp = keras.backend.function(inputs = current_model.layers[0].input, outputs = current_model.layers[-1].output) \
            (np.array(sentence_number).reshape(1,len(sentence_number)))
        actual = vocab_map['capital_' + str(random_capitals[-1])]
        acc_capitals.append(1 if np.argsort(-1 * temp)[0][0] == actual else 0)
        prob_capitals.append(temp[0][vocab_map['capital_' + str(random_capitals[-1])]])
        
    acc_mascots = []
    prob_mascots = []

    for _ in range(n_samples):
        sentence = []
        random_mascots = np.random.choice(np.arange(K), int(L/2), replace = False)
        for random_mascot in random_mascots:
            sentence.append('university_' + str(random_mascot))
            sentence.append('mascot_' + str(random_mascot))
        sentence = sentence[:-1]
        sentence_number = [vocab_map[i] for i in sentence]
        temp = keras.backend.function(inputs = current_model.layers[0].input, outputs = current_model.layers[-1].output) \
            (np.array(sentence_number).reshape(1,len(sentence_number)))
        actual = vocab_map['mascot_' + str(random_mascots[-1])]
        acc_mascots.append(1 if np.argsort(-1 * temp)[0][0] == actual else 0)
        prob_mascots.append(temp[0][vocab_map['mascot_' + str(random_mascots[-1])]])
        

    return sentences, current_model, vocab_map, (np.mean(acc_capitals), np.mean(prob_capitals)), \
                (np.mean(acc_mascots), np.mean(prob_mascots))

In [8]:
K = 10 # number of countries
L = 8 # sentence length
M = 20 # number of words used by each topic
S = 20 # number of words used by both topics
embed_dim = 100 # CBOW embedding dimension
n_sentences = 50000 # number of sentences in the training set
n_samples = 1000

In [10]:
q0 = 0 # probability of having 0 pairs
q1 = 1 # probability of having 1 pair
q2 = 0 # probability of having 2 pairs

accs_c = 0
probs_c = 0
accs_d = 0
probs_d = 0

for _ in range(10):
    sentences, mlm_model, vocab_map, acc_c, acc_d \
        = get_acc_prob(K, M, S, L, q1, q2, embed_dim, n_sentences, n_samples)
    
    print(acc_c)
    print(acc_d)
    
    accs_c += acc_c[0]/10
    probs_c += acc_c[1]/10
    accs_d += acc_d[0]/10
    probs_d += acc_d[1]/10
    
print((accs_c, probs_c))
print((accs_d, probs_d))

(0.0, 1.0961416e-38)
(0.0, 1.716059e-38)
(0.0, 0.0)
(0.0, 4.478327e-39)
(0.0, 1.7272737e-37)
(0.0, 2.0965656e-37)
(0.0, 2.1394542e-38)
(0.0, 6.0655574e-37)
(0.0, 1.0585904e-35)
(0.0, 2.192396e-37)
(0.0, 1.4563822e-37)
(0.0, 1.1323051e-37)
(0.0, 5.5332e-41)
(0.0, 2.676485e-38)
(0.0, 0.0)
(0.0, 0.0)
(0.0, 1.8276212e-37)
(0.0, 0.0)
(0.0, 1.1303424e-37)
(0.0, 1.2668746e-38)
(0.0, 1.123247742065385e-36)
(0.0, 1.2097549239869162e-37)


In [11]:
q0 = 0 # probability of having 0 pairs
q1 = 0 # probability of having 1 pair
q2 = 1 # probability of having 2 pairs

accs_c = 0
probs_c = 0
accs_d = 0
probs_d = 0

for _ in range(10):
    sentences, mlm_model, vocab_map, acc_c, acc_d \
        = get_acc_prob(K, M, S, L, q1, q2, embed_dim, n_sentences, n_samples)
    
    print(acc_c)
    print(acc_d)
    
    accs_c += acc_c[0]/10
    probs_c += acc_c[1]/10
    accs_d += acc_d[0]/10
    probs_d += acc_d[1]/10
    
print((accs_c, probs_c))
print((accs_d, probs_d))

(0.0, 0.00037911593)
(0.0, 0.00022690672)
(0.0, 3.0490983e-07)
(0.0, 2.9735656e-07)
(0.0, 2.7544814e-10)
(0.0, 2.0930548e-09)
(0.0, 0.00091241125)
(0.0, 0.000764921)
(0.0, 0.000105732106)
(0.0, 7.135023e-05)
(0.0, 4.8670736e-05)
(0.0, 3.5429246e-05)
(0.0, 7.824118e-05)
(0.0, 1.3586937e-05)
(0.0, 0.0011388423)
(0.0, 0.0020148247)
(0.0, 8.3345424e-07)
(0.0, 3.6108536e-06)
(0.0, 1.2235369e-06)
(0.0, 3.975696e-08)
(0.0, 0.0002665375694404865)
(0.0, 0.0003130968863912509)


In [12]:
q0 = 1/2 # probability of having 0 pairs
q1 = 1/2 # probability of having 1 pair
q2 = 0 # probability of having 2 pairs

accs_c = 0
probs_c = 0
accs_d = 0
probs_d = 0

for _ in range(10):
    sentences, mlm_model, vocab_map, acc_c, acc_d \
        = get_acc_prob(K, M, S, L, q1, q2, embed_dim, n_sentences, n_samples)
    
    print(acc_c)
    print(acc_d)
    
    accs_c += acc_c[0]/10
    probs_c += acc_c[1]/10
    accs_d += acc_d[0]/10
    probs_d += acc_d[1]/10
    
print((accs_c, probs_c))
print((accs_d, probs_d))

(0.965, 0.74685043)
(0.968, 0.7210558)
(0.838, 0.4809266)
(0.843, 0.51916)
(0.992, 0.8731585)
(0.981, 0.8457265)
(0.909, 0.61087567)
(0.881, 0.5909718)
(0.885, 0.55410486)
(0.888, 0.58143)
(0.975, 0.8083104)
(0.921, 0.6555099)
(0.928, 0.6685761)
(0.961, 0.74516356)
(0.872, 0.54519063)
(0.848, 0.5429051)
(0.969, 0.68875456)
(0.898, 0.6037659)
(0.738, 0.41303924)
(0.898, 0.5754388)
(0.9071000000000001, 0.6389787018299102)
(0.9087, 0.638112735748291)


In [13]:
q0 = 1/2 # probability of having 0 pairs
q1 = 0 # probability of having 1 pair
q2 = 1/2 # probability of having 2 pairs

accs_c = 0
probs_c = 0
accs_d = 0
probs_d = 0

for _ in range(10):
    sentences, mlm_model, vocab_map, acc_c, acc_d \
        = get_acc_prob(K, M, S, L, q1, q2, embed_dim, n_sentences, n_samples)
    
    print(acc_c)
    print(acc_d)
    
    accs_c += acc_c[0]/10
    probs_c += acc_c[1]/10
    accs_d += acc_d[0]/10
    probs_d += acc_d[1]/10
    
print((accs_c, probs_c))
print((accs_d, probs_d))

(1.0, 1.0)
(1.0, 1.0)
(1.0, 1.0)
(1.0, 1.0)
(1.0, 1.0)
(1.0, 1.0)
(1.0, 1.0)
(1.0, 1.0)
(1.0, 1.0)
(1.0, 1.0)
(1.0, 1.0)
(1.0, 1.0)
(1.0, 1.0)
(1.0, 1.0)
(1.0, 1.0)
(1.0, 1.0)
(1.0, 1.0)
(1.0, 1.0)
(1.0, 1.0)
(1.0, 1.0)
(0.9999999999999999, 0.9999999999999999)
(0.9999999999999999, 0.9999999999999999)


In [14]:
q0 = 0 # probability of having 0 pairs
q1 = 1/2 # probability of having 1 pair
q2 = 1/2 # probability of having 2 pairs

accs_c = 0
probs_c = 0
accs_d = 0
probs_d = 0

for _ in range(10):
    sentences, mlm_model, vocab_map, acc_c, acc_d \
        = get_acc_prob(K, M, S, L, q1, q2, embed_dim, n_sentences, n_samples)
    
    print(acc_c)
    print(acc_d)
    
    accs_c += acc_c[0]/10
    probs_c += acc_c[1]/10
    accs_d += acc_d[0]/10
    probs_d += acc_d[1]/10
    
print((accs_c, probs_c))
print((accs_d, probs_d))

(1.0, 0.99840504)
(1.0, 0.9982628)
(1.0, 0.9979707)
(1.0, 0.9987805)
(1.0, 0.99700636)
(1.0, 0.99666184)
(1.0, 0.99446255)
(1.0, 0.99904746)
(1.0, 0.9966239)
(1.0, 0.998833)
(1.0, 0.9974218)
(1.0, 0.9973606)
(1.0, 0.9982404)
(1.0, 0.99438524)
(1.0, 0.9977408)
(1.0, 0.99812275)
(1.0, 0.99599814)
(1.0, 0.9974061)
(1.0, 0.99858785)
(1.0, 0.9986317)
(0.9999999999999999, 0.9972457528114319)
(0.9999999999999999, 0.9977492034435272)


In [15]:
q0 = 1/3 # probability of having 0 pairs
q1 = 1/3 # probability of having 1 pair
q2 = 1/3 # probability of having 2 pairs

accs_c = 0
probs_c = 0
accs_d = 0
probs_d = 0

for _ in range(10):
    sentences, mlm_model, vocab_map, acc_c, acc_d \
        = get_acc_prob(K, M, S, L, q1, q2, embed_dim, n_sentences, n_samples)
    
    print(acc_c)
    print(acc_d)
    
    accs_c += acc_c[0]/10
    probs_c += acc_c[1]/10
    accs_d += acc_d[0]/10
    probs_d += acc_d[1]/10
    
print((accs_c, probs_c))
print((accs_d, probs_d))

(1.0, 1.0)
(1.0, 1.0)
(1.0, 1.0)
(1.0, 1.0)
(1.0, 1.0)
(1.0, 1.0)
(1.0, 1.0)
(1.0, 1.0)
(1.0, 1.0)
(1.0, 1.0)
(1.0, 1.0)
(1.0, 1.0)
(1.0, 1.0)
(1.0, 1.0)
(1.0, 1.0)
(1.0, 1.0)
(1.0, 1.0)
(1.0, 1.0)
(1.0, 0.9999999)
(1.0, 1.0)
(0.9999999999999999, 0.999999988079071)
(0.9999999999999999, 0.9999999999999999)
